# Collective progress

**Example notebook for creating anonymised, collective information on progress**

* Before running this notebook, you need to prepare the data you want to assess. To do so, please use the notebooks  "prepare-PRIMAP-hist-data-for-collective-progress-plots.ipynb" or "prepare-PRIMAP-data.ipynb" in case the data you need is not yet available.
* For testing, some example data is available in the folder "proc_data"
* Enter the name of the file that you wish to use in the second cell and some of the plotting parameters. After that you can run the full notebook with minimal changes. 

In [7]:
# import modules

# system 
import re
import os

# calculation
import pandas as pd
import numpy as np

# plotting
%matplotlib inline
import seaborn
import matplotlib

# global stocktake tools
from gst_tools.make_plots import *
import gst_tools.gst_utils as utils


In [27]:
# USER INPUT

# First, choose which file you want to plot the data for
data_file_name = 'PRIMAP-hist_UN-2017_calc_CO2-total-excl-LU-per-population.csv'


# other options include...
# 'PRIMAP-hist_v2.0_KyotoGHG-AR4-total-excl-LU.csv'
# 'UN-population-data-2017.csv'
# 'PRIMAP-hist_v2.0_Energy-CO2.csv'
# 'PRIMAP-hist_UN-2017_calc__CO2-per-population.csv'
# 'PRIMAP-hist_UN-2017_calc_CO2-total-excl-LU-per-population.csv'
# 'WDI2017_GDP-PPP.csv'

# Second, choose which years you are interested in analysing
years_of_interest = ['1990', '2000', '2015']

# Third, update data description display names!
# TODO - default to automatic if not specified.
variable_name_to_display = 'per capita CO2 emissions'
data_source_to_display = 'PRIMAP-hist v2.0; UN'

# Save plots?
# Set the following to True if plots should be saved. 
# If False, plots will be shown on screen but not saved to a file.
save_opt = True

In [28]:
# DATA READING AND PREP

# read the data from file 
fname_in = os.path.join('proc-data', data_file_name)
data = pd.read_csv(fname_in)

# Check the data format
if not utils.verify_data_format(data):
    print('WARNING: The data is not correctly formatted! Please check before continuing!')

# extract the key information
variable = data['variable'].unique()[0]
unit = data['unit'].unique()[0]

# tidy up for next stesps
data_years = utils.set_countries_as_index(data)
data_years = data_years.dropna(axis=1, how='any')

# remove comment below to display the data
#data_years

In [29]:
# Plot 1 - make a histogram of absolute data

# how should the xaxis be labelled?
xaxlabel = variable_name_to_display

# make a plot for each year
for selected_year in years_of_interest:
    
    title = 'distribution of ' + variable_name_to_display + ' in ' + str(selected_year)
    
    make_histogram(data_years[selected_year], unit,
                   xlabel=xaxlabel, title=title, 
                   sourcename=data_source_to_display,
                   remove_outliers=True, ktuk=3,
                   save_plot=save_opt, plot_name=(variable + '-' + 'absolute' + '-' + str(selected_year)))


---------
Making  CO2-total-excl-LU-per-population-absolute-1990 plot.
---------
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 1990, dtype: float64)
upper outliers are: 
country
LUX    31.168886
QAT    28.754631
Name: 1990, dtype: float64
---
---------
Making  CO2-total-excl-LU-per-population-absolute-2000 plot.
---------
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 2000, dtype: float64)
upper outliers are: 
country
ARE    33.281298
BHR    30.694508
KWT    27.160914
QAT    64.835623
TTO    25.473508
Name: 2000, dtype: float64
---
bins set to range(0, 23)
---------
Making  CO2-total-excl-LU-per-population-absolute-2015 plot.
---------
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 2015, dtype: float64)
upper outliers are: 
country
ARE    30.040521
BRN    22.752202
KWT    25.661912
QAT    53.595773
TTO    49.555471
Name: 2015, dtype: float64
---
bins set to range(0, 21)


In [36]:
# Plot 2 - trends

# Calculate trends from the absolute data
# trends - % change in any given year
# rolling_trends - % annual change averaged over the specifiied number of years (here 5 is recommended)
# trends_unit - unit of the trend, here % change
trends, rolling_trends, trends_unit = utils.calculate_trends(data_years, num_years_trend=5)

# define some labels for the plots
trends_variable = 'average annual change' 
thistitle = "5-year rolling average trend in \n" + variable_name_to_display + "\nin " + str(data_years.columns[-1])

# make a plot showing the trend in the final year of available data
make_histogram(rolling_trends.iloc[:,-1], trends_unit, 
               xlabel=trends_variable,
               title=thistitle,
               remove_outliers=True, ktuk=3,
               sourcename=data_source_to_display, 
               save_plot=save_opt, plot_name=(variable + '-' + 'rolling-average'))


Averaging trend over 5 years.
---------
Making  CO2-total-excl-LU-per-population-rolling-average plot.
---------
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 2015, dtype: float64)
upper outliers are: 
country
MMR    14.710766
MOZ    22.507771
NIU    31.461331
Name: 2015, dtype: float64
---
bins set to range(-11, 11)


In [31]:
# Plot 3 - change since year X

# run calculations - dataframe of differences in all years relative to the specified year
# the function returns both absolute and relative (%) values
df_abs_diff_1990, df_perc_diff_1990 = utils.calculate_diff_since_yearX(data_years, '1990')
df_abs_diff_2005, df_perc_diff_2005 = utils.calculate_diff_since_yearX(data_years, '2005')

# make plots

# a few selected years; difference from 1990
for selected_year in years_of_interest:
    make_histogram(df_perc_diff_1990[selected_year], "%", 
                   xlabel='change since 1990', 
                   title=('change in ' + variable_name_to_display + '\n from 1990 to ' + str(selected_year)), 
                   sourcename=data_source_to_display,
                   remove_outliers=True, ktuk=3, 
                   save_plot=save_opt, plot_name=(variable + '-' + 'change-since-1990' + '-in-' + str(selected_year)))

    
# decrease in the last year as compared to 2005 
make_histogram(df_perc_diff_2005.iloc[:,-1], '%', 
               xlabel='change since 2005', 
               title=('change in ' + variable_name_to_display + '\n from 2005 to ' + str(df_perc_diff_2005.columns[-1])), 
               sourcename=data_source_to_display,
               remove_outliers=False, 
               save_plot=save_opt, plot_name=(variable + '-' + 'change-since-2005'))

# same plot, but removing outliers to show difference...
make_histogram(df_perc_diff_2005.iloc[:,-1], '%', 
               xlabel='change since 2005', 
               title=('change in ' + variable_name_to_display + '\n from 2005 to ' + str(df_perc_diff_2005.columns[-1])), 
               sourcename=data_source_to_display,
               remove_outliers=True, 
               save_plot=save_opt, plot_name=(variable + '-' + 'change-since-2005-excl-outliers'))


Calculating difference compared to 1990
Calculating difference compared to 2005
---------
Making  CO2-total-excl-LU-per-population-change-since-1990-in-1990 plot.
---------
---------
All values in the series are the same! Exiting plotting routine for CO2-total-excl-LU-per-population-change-since-1990-in-1990
---------
---------
Making  CO2-total-excl-LU-per-population-change-since-1990-in-2000 plot.
---------
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 2000, dtype: float64)
upper outliers are: 
country
BTN    187.340895
GNQ    354.003929
LAO    232.243729
NPL    239.987233
SYC    201.055769
TLS    284.729624
Name: 2000, dtype: float64
---
bins set to range(-150, 150, 15)
---------
Making  CO2-total-excl-LU-per-population-change-since-1990-in-2015 plot.
---------
-----------
Identifying and removing outliers
lower outliers are:
Series([], Name: 2015, dtype: float64)
upper outliers are: 
country
BTN     473.181642
GNQ    3722.391280
NPL     585.1902

## Below here is space for code for testing and debugging!

In [7]:
# show example data
data.columns


Index(['variable', 'country', 'unit', '1990', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015'],
      dtype='object')